In [ ]:
%%html
<style>
body {
    font-family: "Avenir", cursive, sans-serif;
}
</style> 


# First-Order Splitting Patterns in <sup>1</sup>H NMR Spectra, Part 1: The "*n* + 1" Rule

### **Instructions:** From the "Cell" dropdown menu, select  "Run All". The interactive plots that accompany the text will be activated.

In [ ]:
from IPython.display import HTML

In [ ]:
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
The python code for this notebook is hidden by default. 
To toggle on/off the raw code, click <a href="javascript:code_toggle()">here</a>.''')

This tutorial assumes the user is familiar with NMR spectroscopy at least up to the meaning of chemical shifts and signal integrations. Familiarity with the "n + 1" rule is helpful, but this will be covered in the tutorial.

In [ ]:
import numpy as np

In [ ]:
import holoviews as hv
import panel as pn
hv.extension('bokeh', width=100)
pn.extension()

In [ ]:
from nmrsim.firstorder import multiplet
from nmrsim.math import add_lorentzians


## What Is "First-Order" Behavior?

To accurately model NMR spectra, quantum mechanical calculations are required. This is not something that can be done "on the back of an envelope" with a calculator and ruler. However, it is often possible to adequately interpret the spectra using a simpler physical model. We will describe and use this model in this tutorial. Spectra that closely follow this simpler model are said to follow **first-order** behavior. The explanations that follow use this simplistic physical model.

When the observed spectra cannot adequately be interpreted with this approximate model, they are said to follow **second-order** behavior, and more complicated calculations are required to interpret them accurately. We will see an example of this after we introduce the concept of *J*-coupling, starting with the "doublet" pattern.

In [ ]:
def lineshape_from_peaklist(peaklist, w=0.5, points=800, limits=None):
    """

    Parameters
    ----------
    peaklist : [(float, float)...]
        A list of (frequency, intensity) tuples
    w : float
        Peak width at half height (Hz)
    points : int
        The number of data points in the lineshape
    limits : (float, float)
        The frequency (left/right) limits for the lineshape

    Returns
    -------
    ([float...], [float...])
        The lists of x coordinates (frequency) and corresponding y coordinates (intensity) for the lineshape.
    """
    peaklist.sort()
    if limits:
        try:
            l_limit, r_limit = limits
            l_limit = float(l_limit)
            r_limit = float(r_limit)
        except Exception as e:
            print(e)
            print('limits must be a tuple of two numbers')
            raise
        if l_limit > r_limit:
            l_limit, r_limit = r_limit, l_limit
    else:
        l_limit = peaklist[0][0] - 50
        r_limit = peaklist[-1][0] + 50
    x = np.linspace(l_limit, r_limit, points)
    y = add_lorentzians(x, peaklist, w)
    return x, y

In [ ]:
def n_plus_one(J, n, v=100, i = 1, w=0.5):
    singlet = (v, i)  # center at 100 Hz; intensity 1
    couplings = [(J, n)]
    peaklist = multiplet(singlet, couplings)
    x, y = lineshape_from_peaklist(peaklist, w=w)
    return hv.Curve(zip(x, y)).options(axiswise=True, invert_xaxis=True)

## *J* Couplings

Spin-½ nuclei can be considered to be "spin-up" ("α"; aligned with the spectrometer's magnetic field) or "spin-down" ("β"; aligned against the spectrometer's magnetic field). The odds of it being either spin-up or spin-down are very nearly equal. 

The small magnetic moments of neighboring nuclei alter the net magnetic field strengh that a nucleus experiences. If a nucleus has one such neighbor, there's close to a 50% chance that it will experience a slightly stronger total magnetic field, and a 50% chance of a slightly weaker magnetic field. That means that half the time the nucleus will resonate at a slightly higher frequency, and half the time at a slightly lower frequency. The two nuclei are said to be **coupled**, and the size of the frequency difference between the high- and low-frequency peaks in Hz is referred to as the **coupling constant**, or ***J*-value**.

You can simulate this behavior using the plot below. In the absence of *J*-coupling, a single peak or **singlet** is seen. Check the box to toggle the *J* coupling to one neighbor on and off. Turning the coupling on changes the singlet to a two-peak **doublet**.



In [ ]:
def toggle_doublet(coupled=False):
    n = 1 if coupled else 0
    return n_plus_one(10.0, n, w=0.5).redim(y=hv.Dimension('intensity', range=(-0.1, 1.1)),
                                           x=hv.Dimension('𝜈 (Hz)', range=(120, 80)))


toggle_doublet_app = pn.interact(toggle_doublet, n=False)

toggle_doublet_app

Note that when the coupling is turned on, the intensities of the two peaks of the resulting doublet are half that of the original singlet. 
The signal appears as a singlet at 100 Hz in the absence of coupling, but two signals at 95 and 105 Hz in the presence of a 10-Hz *J* coupling.

The simulations in this tutorial will use Hz as the x-axis, and $\nu$ (nu) to represent a frequency in Hz. A signal at 100 Hz would appear at 100 Hz higher frequency than the signal for TMS (tetramethylsilane). In standard NMR spectra, the $\delta$/ppm (parts per million) scale is used instead. For <sup>1</sup>H NMR spectra, if you divide the frequency of the signal in Hz by the frequency of the spectrometer, you get the chemical shift in ppm. A 100-Hz signal would be 1 ppm on a 100 MHz spectrometer, but 0.25 ppm on a 400 MHz spectrometer (the standard spectrometer we use for CHEM333 lab samples).

## Two Coupled Nuclei

Before discussing sets of coupled nuclei, we need to distinguish between nuclei that are *chemical shift-equivalent* vs. those that are *chemical shift non-equivalent*.  

### Chemical Shift Equivalence

If two nuclei exhibit the exact same chemical shift, either because of the molecule's symmetry, or because of a rapid exchange process, they are said to be **chemical shift equivalent**. 

Chloroethane (Cl-CH<sub>2</sub>-CH<sub>3</sub>) has two sets of chemical shift equivalent protons. The protons of the methyl group are equivalent because rapid rotation about the C-C bond exchanges them, and on average they will have the same chemical shift:

<img src='img/homotopic.png' style='width: 600px;'>

The protons of the downfield methylene group are chemical shift-equivalent because there is a mirror plane of symmetry relating them:

<img src='img/enantiotopic.png' style='width: 600px;'>

***Splitting is not observed between chemical shift-equivalent nuclei***. For example, the methyl group of chloromethane would appear as a singlet. It is often incorrectly said that "nuclei with the same chemical shift don't couple to each other", but that's not true--they do couple, but you won't see extra peaks appear in the NMR as a consequence.

If two spin-½ nuclei, with different chemical shifts, are coupled to each other and nothing else, their signals will each be split into a doublet. If their chemical shift separation is large (e.g. dichloroacetaldehyde), each will appear as a 1:1 doublet, and the chemical shift of the signal can be measured from the midpoint of the doublet. Here is a first-order simulation for two coupled nuclei, at 100 Hz and 500 Hz, with a 10-Hz coupling:

In [ ]:
def two_doublets(v1, v2, J, w=0.5, points=10000):
    peak1 = (v1, 1)  
    couplings = [(J, 1)]
    peak2 = (v2, 1)
    peaklist = multiplet(peak1, couplings) + multiplet(peak2, couplings)
    x, y = lineshape_from_peaklist(peaklist, w=w, points=points)
    return hv.Curve(zip(x, y)).options(axiswise=True, invert_xaxis=True,
                                      xlabel='𝜈',
                                      ylabel='intensity')

In [ ]:
two_doublets(100, 500, 10)

### Pople Nomenclature

Groups of coupled nuclei, or **spin systems**, can be described using the **Pople Nomenclature** system, which we introduce with this example. This system uses a different upper-case letter to refer to each set of chemical shift-equivalent nuclei. It's traditional to use the letters A and X for two sets of chemical-shift equivalent nuclei with very different chemical shifts (i.e. $\Delta\nu >> J$). The spin system described above (two doublets) would be an example of an **AX** spin system. Subscripts are added if there are multiple nuclei of the same type; for example, the spin system for chloroethane can be described as A<sub>2</sub>X<sub>3</sub>. 

***J-splitting is reciprocal:*** If H<sub>A</sub> is split by H<sub>X</sub> with *J* = 10 Hz, then H<sub>X</sub> is split by H<sub>A</sub> with a 10-Hz splitting as well. 

## First-Order vs. Second-Order Behavior



As the chemical shift difference between the two doublets decreases, the appearance of the signals deviates from the first-order approximation described above (1:1 doublets, with the chemical shift being the midpoint between the two peaks). The size of the inner peak of each doublet grows larger, and the outer peaks smaller. This effect where the signals appear to "lean towards each other" is often described as "tenting" or "roofing". Also, the chemical shift for each nucleus is no longer the midpoint of the 1:1 doublet, but closer to the larger peak than the smaller. When the the distortions are severe enough that a first-order analysis is insufficiently accurate (roughly when $\Delta\nu < 5 J$, the spin system would be considered second-order and described as **AB** rather than AX. The proximity of the letters A and B in the alphabet implies the proximity of the two signals in the NMR spectrum.

This behaviour can be explored using the interactive plot below:

In [ ]:
from nmrsim.discrete import AB

In [ ]:
def ab_wrapper(v1, v2, J):
    vab = abs(v2 - v1)
    vcentr = (v1 + v2) / 2
    peaklist = AB(J, vab, vcentr)
    return peaklist

In [ ]:
def ab_distortion(v1 = 100, v2 = 500):
# Placeholders for the HTML codes for nu_1 and nu_2
# &nu;&#x2081;
# &nu;&#x2082;
    v1_arrow = hv.Arrow(v1,0, 'ν₁', '^')
    v2_arrow = hv.Arrow(v2,0, 'ν₂', '^')
    datapoints = max(int(abs(v2 - v1) * 100), 800)  # sufficient datapoints to mitigate inaccurate intensities and "jittering"

    coupled = lineshape_from_peaklist(ab_wrapper(v1, v2, 10), points=datapoints)
    plot = hv.Curve(zip(*coupled)) * v1_arrow * v2_arrow
    return plot.options(axiswise=True, invert_xaxis=True,
                       xlabel='𝜈').redim(y=hv.Dimension('intensity', range=(-0.4, 1.2)))

In [ ]:
ab_distortion_interactive = pn.interact(ab_distortion, v1=(0, 600, 0.1, 100), v2=(0, 600, 0.1, 500))

In [ ]:
ab_distortion_interactive

- As you drag the frequency sliders closer to each other, you will see the "roofing" effect become more pronounced. The arrows indicating the frequencies for each nucleus will also move away from the midpoint of each signal and closer to the larger, inner peak. When the difference between $\nu_1$ and $\nu_2$ is ~ 17 Hz, the signal could easily be mistaken for an *n* + 1 "quartet" (see below), and an AB signal with clear second-order behavior is often referred to as an **AB quartet**.
- When the difference between $\nu_1$ and $\nu_2$ is ~ 4 Hz, the outer peaks are very small and could easily be missed if the baseline was noisy or if other signals were nearby. The two inner peaks are starting to collapse together. This could easily be misidentified as a ~2 proton doublet, since the outer peaks don't contribute much to the total area. The small distance between the two inner peaks could be misinterpreted as a small J coupling.

## Origin of the "n + 1" Rule

The **n + 1 rule** predicts that, if a nucleus is being split by *n* nuclei by a coupling constant *J*, its NMR signal will have *n* + 1 peaks. We have already discussed the *n* = 0 case, where a singlet is seen in the absence of coupling, and the *n* = 1 case, where a doublet is seen when the nucleus is coupled to one neighbor. Additional splittings give rise to additional peaks (triplet, quartet, etc.) The following table summarizes the nomenclature for these signals, as well as their commonly-used abbreviations.

| n | # of peaks | name | abbreviation |
|:-:|:-:|:-:|:-:|
| 0 | 1 | singlet |s|
| 1 | 2 | doublet |d|
| 2 | 3 | triplet |t|
| 3 | 4 | quartet |q|
| 4 | 5 | quintet |quint|
| 5 | 6 | sextet | |
| 6 | 7 | septet | |
| 7 | 8 | octet | |
| 8 | 9 | nonet | |

A neighboring spin-½ nuclei (e.g <sup>1</sup>H; <sup>13</sup>C) nucleus has close to a 50:50 chance of being spin-up vs. spin-down, with each state adding or subtracting to the overall strength of the magnetic field experienced by the nucleus of interest. The statistical chance of the nucleus of interest experiencing a certain magnetic field strength (and thus a certain frequency/chemical shift) resembles the statistical chance of head/tails coin flips. If you flip two coins, the possible results are:

* both heads: 25%
* one heads, one tails: 50%
* both tails: 25%

or 1 : 2 : 1. 

Similarly, if a proton has two proton neighbors, each of which can be either α or β, the odds are close to:

* both α: 25%
* one α, one β: 50 %
* both β: 25%

or 1 : 2 : 1. 

For the plot below, drag the slider to change the number of neighbors n (each with a 10-Hz coupling) from 0 to 1 to 2. When n = 2, a "triplet" pattern with 1 : 2 : 1 intensities results. Compared to the intensity of the singlet in the absence of coupling, the triplet intensities are 0.25 : 0.5 : 0.25. Note that the integration of this signal will be 1 regardless of the number of splittings.

In [ ]:
def singlet_to_triplet(n):
    return n_plus_one(J=10.0, n=n, w=0.5).redim(
        y=hv.Dimension('intensity', range=(-0.1, 1.1))
    ).opts(xlabel='𝜈')

In [ ]:
singlet_to_triplet_app = pn.interact(singlet_to_triplet, n=(0, 2, 1, 0))

In [ ]:
singlet_to_triplet_app

As *n* increases, the relative intensities of the peaks in the signal are predicted by [Pascal's triangle](https://en.wikipedia.org/wiki/Pascal%27s_triangle):

<img src='img/pascal_triangle.png' style='width: 300px;'>

Use the slider below to change *n* from 0 through 8 to see what the corresponding *n* + 1 multiplet looks like:

In [ ]:
def simple_multiplet(n):
    return n_plus_one(J=7.0, n=n, w=0.5).redim(
        y=hv.Dimension('intensity', range=(-0.1, 1.1))
    ).opts(xlabel='𝜈')

In [ ]:
simple_multiplet_app = pn.interact(simple_multiplet, n=(0, 8, 1, 1))


In [ ]:
simple_multiplet_row = pn.Row(simple_multiplet_app, width_policy='max')


In [ ]:
simple_multiplet_row

Note that as *n* increases, the size difference between the outermost and innermost peaks increases as well. At higher values of *n*, it can be hard to see the outermost peaks. In real spectra with poor signal-to-noise ratio, they can easily be lost in the noise. Without accurate intensities for the individual peaks, it can be easy to misinterpret such a signal (e.g. interpret a nonet as a septet).

- For the above simulation, set *n* to 8. If you only see 7 peaks instead of 9, use one of the zoom tools in the right menu bar (indicated by the magnifying glass in the icon) to expand or enlarge the signal and see all 9 peaks.

***n + 1* multiplets are seen when the couplings *J* are all the same size**. When the magnitudes of the couplings to neighboring nuclei are different, more complicated patterns are seen. These will be covered in [the next tutorial](first_order_part_2.ipynb).